In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Pais;

CREATE TABLE Pais (
  nome VARCHAR(60),            -- eliminar ambiguidade!
  sigla VARCHAR(3) NOT NULL,
  regiao VARCHAR(30),
  classificacaoRenda VARCHAR(30),
  Gini INTEGER,
  PIBperCapita DOUBLE,
  IDH INTEGER,
  populacao INTEGER,
  PRIMARY KEY(sigla)
) AS SELECT
    country,
    label,
    region,
    wb_class,
    gini,
    gdbPPPperCapita,
    hdi,
    populationTotal
FROM CSVREAD('../../../data/Filtered Data/countries.csv');

DROP TABLE IF EXISTS DST;
CREATE TABLE DST (
  regiao VARCHAR(30),
  ano INTEGER,
  quantidade DOUBLE,
  PRIMARY KEY(regiao, ano)
) AS SELECT
    region,
    year,
    nominal
FROM CSVREAD('../../../data/Filtered Data/infections.csv');

In [3]:
SELECT TOP 3 * FROM Pais;
SELECT TOP 3 * FROM DST;

In [4]:
CREATE VIEW RegiaoxInfeccoes AS
SELECT DP.ano, P.regiao, 
       CAST(SUM(DP.quantidade) AS DOUBLE) / CAST(SUM(DG.quantidade) AS DOUBLE)*100.0 Infeccoes
FROM Pais P, DST DP, DST DG
WHERE P.sigla = DP.regiao 
      AND DG.ano = DP.ano 
      AND DP.quantidade IS NOT NULL
GROUP BY P.regiao, DP.ano
ORDER BY DP.ano,
         P.regiao;

SELECT * FROM RegiaoxInfeccoes;

CALL CSVWRITE('../../saida/regiao-x-infeccoes.csv', 'SELECT * FROM RegiaoxInfeccoes');